- Since 01/19/2019
    - Half of all large down spikes (2+ std moves) have occurred before 11 am
    - A quarter of all large up spikes (2+ std moves) have occurred after 3 pm
    - **Avoid trying to defend call spreads after 3 pm**
    - **Avoid trying to defend put spreads before 11 am**

In [47]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import math
def time_block_strtoactual(i):
    curr_time_block = dt.datetime.today().replace(hour = 9, minute = 30, second = 0, microsecond = 0) + dt.timedelta(minutes = 30*(i-1))
    return curr_time_block.strftime("%H:%M")

In [103]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def time_remaining(weekly_options, curr_spx):

    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')

def get_spreads(weekly_options, curr_spx, curr_vix, curr_skew, expiry_index = 0, rowshift = 1, commissions = 0.0266):

    expirations_dates = weekly_options.Expiry.drop_duplicates().tolist()
    
    weekly_options = weekly_options[weekly_options.Expiry == expirations_dates[expiry_index]]

    typ = weekly_options.reset_index().loc[0,'Type']
    
    if typ == 'P':
        weekly_options = weekly_options.sort_values('Strike', ascending = False).reset_index(drop = True)
    if typ == 'C':
        weekly_options = weekly_options.sort_values('Strike', ascending = True).reset_index(drop = True)
        
    time_remaining = weekly_options.reset_index(drop = True).loc[0,'time_remaining']

    shorts = weekly_options[['Bid', 'Strike', 'bidDelta', 'bidGamma', 'bidTheta', 'bidVega']]
    shorts.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']
    longs = weekly_options[['Ask','Strike', 'askDelta', 'askGamma','askTheta', 'askVega']]
    longs.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']

    spreads = shorts - longs.shift(-rowshift)
    spreads['Short_Strike'] = shorts.Strike
    spreads['Short_Delta'] = shorts.Delta
    spreads['Long_Strike'] = longs.Strike.shift(-rowshift)
    spreads = spreads[['Short_Strike', 'Long_Strike', 'Short_Delta', 'Price',
                       'Strike', 'Delta', 'Gamma', 'Theta', 'Vega']]

    spreads['Credit'] = spreads.Price - commissions

    if typ == 'P':
        spreads['MaxLoss'] = (-spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] - spreads.Credit
    if typ == 'C':
        spreads['MaxLoss'] = (spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] + spreads.Credit

    del spreads['Price'], spreads['Strike']

    spreads = spreads[spreads.Credit > 0].dropna().reset_index(drop = True)

    stepsize = 0.01
    period_iv = np.sqrt(time_remaining)*curr_vix/100
    
    skew_implied_2std = 0.027*(curr_skew - 100)/10
    skew_implied_3std = 0.006*(curr_skew - 100)/10

    period_downside_iv_2std = (-2*period_iv)/norm.ppf(skew_implied_2std,0,1)
    period_downside_iv_3std = (-3*period_iv)/norm.ppf(skew_implied_3std,0,1)

    spreads['EV'] = np.nan
    spreads['Win Prob'] = np.nan

    ev_lsts = []

    for idx, row in spreads.iterrows():

        if typ == 'P':
            ev_df = pd.DataFrame({'SPX': np.arange(row.Long_Strike, row.Short_Strike + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row.Short_Strike + stepsize)]
            ev_df['PnL'] = ev_df['SPX'] - row.Short_Strike + row.Credit
        if typ == 'C':
            ev_df = pd.DataFrame({'SPX': np.arange(row.Short_Strike, row.Long_Strike + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row.Long_Strike + stepsize)]
            ev_df['PnL'] = row.Short_Strike - ev_df['SPX'] + row.Credit

        ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0, period_iv)
        ev_df[ev_df.SPX < curr_spx*(1 - period_iv)]['Prob'] = norm.cdf(ev_df[ev_df.SPX < curr_spx*(1 - period_iv)]['SPX']/curr_spx - 1,
                                                                       0, period_downside_iv_2std)
        ev_df[ev_df.SPX < curr_spx*(1 - 2*period_iv)]['Prob'] = norm.cdf(ev_df[ev_df.SPX < curr_spx*(1 - 2*period_iv)]['SPX']/curr_spx - 1,
                                                                         0, period_downside_iv_3std)
        
        lb = ev_df.loc[0,'Prob']
        ub = ev_df.loc[len(ev_df) - 1,'Prob']
        ev_df['Prob'] = ev_df['Prob'].diff()
        ev_df.loc[0,'Prob'] = lb
        ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub
        ev_df['EV'] = ev_df.Prob*ev_df.PnL

        ev_lsts.append(ev_df)
        total_ev = sum(ev_df['EV'])
        win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])

        spreads.loc[idx, 'EV'] = total_ev
        spreads.loc[idx, 'Win Prob'] = win_prob
    
    spreads['Expiry'] = expirations_dates[expiry_index]
    spreads.Expiry = pd.to_datetime(spreads['Expiry'])
    return spreads, ev_lsts


#%% Stats
def get_block_stats(curr_block, block_label):

    curr_spx_trend = curr_block.close_spx - curr_block.reset_index(drop = True).loc[0,'close_spx']
    curr_vix_trend = curr_block.close_vix - curr_block.reset_index(drop = True).loc[0,'close_vix']
    spx_X = curr_spx_trend.index ## X usually means our input variables (or independent variables)
    spx_Y = curr_spx_trend ## Y usually means our output/dependent variable

    vix_X = curr_vix_trend.index ## X usually means our input variables (or independent variables)
    vix_Y = curr_vix_trend ## Y usually means our output/dependent variable

    # # Note the difference in argument order
    spx_ols = sm.OLS(spx_Y, spx_X).fit().summary()
    spx_ols_r2 = float(pd.read_html(spx_ols.tables[0].as_html())[0].loc[1,3])
    spx_ols_coeff = float(pd.read_html(spx_ols.tables[1].as_html())[0].loc[1,1])

    vix_ols = sm.OLS(vix_Y, vix_X).fit().summary()
    vix_ols_r2 = float(pd.read_html(vix_ols.tables[0].as_html())[0].loc[1,3])
    vix_ols_coeff = float(pd.read_html(vix_ols.tables[1].as_html())[0].loc[1,1])

    spx_bar_std = np.std(curr_block.close_spx/curr_block.open_spx)
    vix_bar_std = np.std(curr_block.close_vix/curr_block.open_vix)

    spx_block_std = np.std((curr_block.close_spx.shift(1)/curr_block.close_spx).dropna())
    vix_block_std = np.std((curr_block.close_vix.shift(1)/curr_block.close_vix).dropna())

    spx_adf_sig = ts.adfuller(np.log(curr_block.close_spx))[1]
    vix_adf_sig = ts.adfuller(np.log(curr_block.close_vix))[1]

    spx_block_return = curr_block.loc[len(curr_block) - 1, 'close_spx']/curr_block.loc[0, 'close_spx'] - 1
    vix_block_return = curr_block.loc[len(curr_block) - 1, 'close_vix']/curr_block.loc[0, 'close_vix'] - 1
    
    spx_vix_corr = np.corrcoef(curr_block.close_spx,curr_block.close_vix)[0,1]
    
    block_stats = pd.DataFrame({'Block_Label': block_label,
                                'SPX_OLS_Coeff': spx_ols_coeff, 
                                'SPX_OLS_R2': spx_ols_r2,
                                'SPX_bar_std': spx_bar_std,
                                'SPX_block_std': spx_block_std,
                                'SPX_adf_sig': spx_adf_sig,
                                'SPX_block_return': spx_block_return,
                                'VIX_OLS_Coeff': vix_ols_coeff, 
                                'VIX_OLS_R2': vix_ols_r2,
                                'VIX_bar_std': vix_bar_std,
                                'VIX_block_std': vix_block_std,
                                'VIX_adf_sig': vix_adf_sig,
                                'VIX_block_return': vix_block_return,
                                'SPX_VIX_corr': spx_vix_corr,
                                'VIX_open': curr_block.loc[0,'close_vix']}, index = [0])
    return block_stats

def day_block_stats(curr_data):
    block_label = 1
    day_timeblock_stats = []
    for time_block in range(0,len(curr_data),30):
        if time_block < len(curr_data):
            curr_block = curr_data.loc[time_block:time_block + 30, :].reset_index(drop = True)
            day_timeblock_stats.append(get_block_stats(curr_block, "Time{}".format(block_label)))
        block_label += 1

    day_timeblock_stats = pd.concat(day_timeblock_stats, axis = 0).reset_index(drop = True)

    return day_timeblock_stats
#%%


In [200]:
client_id = 12

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

spx = Index('SPX', 'CBOE')
vix = Index('VIX', 'CBOE')
skew = Index('SKEW', 'CBOE')
ib.qualifyContracts(skew)
ib.qualifyContracts(spx)

ib.reqHeadTimeStamp(spx, whatToShow='TRADES', useRTH=True)
ib.reqHeadTimeStamp(vix, whatToShow='TRADES', useRTH=True)

end_date = '' #'20100506 13:00:00 PST'
duration = '1 D'

spx_bars = ib.reqHistoricalData(
        spx,
        endDateTime=end_date,
        durationStr=duration,
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1)

vix_bars = ib.reqHistoricalData(
        vix,
        endDateTime=end_date,
        durationStr=duration,
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1)

spx_df = util.df(spx_bars).set_index('date')
spx_df.index = pd.to_datetime(spx_df.index)

vix_df = util.df(vix_bars).set_index('date')
vix_df.index = pd.to_datetime(vix_df.index)

curr_spx = spx_df.reset_index().loc[len(spx_df) - 1, 'close']

curr_vix = vix_df.reset_index().loc[len(vix_df) - 1, 'close']

curr_skew = ib.reqMktData(skew, '', False, False).marketPrice()

if math.isnan(curr_skew):
    skew = pd.read_csv('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/skewdailyprices.csv', 
                       skiprows = 1)[['Date','SKEW']].set_index('Date')
    curr_skew = skew.reset_index().loc[len(skew) - 1, 'SKEW']

chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

chains_df = util.df(chains)
chain = next(c for c in chains if c.tradingClass == 'SPXW' and c.exchange == 'SMART')



In [201]:
bounds = curr_spx*(curr_vix/(100*np.sqrt(52)))

strikes = [strike for strike in chain.strikes
           if strike % 5 == 0
           and curr_spx - bounds < strike < curr_spx + bounds]


all_expirations = sorted(exp for exp in chain.expirations)
expirations = all_expirations[4:6]
expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in expirations]
expirations

['20190301', '20190304']

In [202]:
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART')
        for right in rights
        for expiration in expirations
        for strike in strikes]

ib.qualifyContracts(*contracts)

spx_options = ib.reqTickers(*contracts)

puts_rows = []
calls_rows = []

for curr_option in spx_options:
    try:
        curr_row = create_dfrow(curr_option)
        if curr_option.contract.right == 'P':
            puts_rows.append(curr_row)
        elif curr_option.contract.right == 'C':
            calls_rows.append(curr_row)
    except:
        continue

puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])

puts = time_remaining(puts, curr_spx)
calls = time_remaining(calls, curr_spx)

curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
puts.to_csv('ib_puts' + curr_time + '.csv')
calls.to_csv('ib_calls' + curr_time + '.csv')


ib.disconnect()

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [203]:
reg_estimates = pd.read_csv('reg_estimates.csv', index_col = 0)

index_intraday = spx_df.join(vix_df, lsuffix = '_spx', rsuffix = '_vix').reset_index()
#index_intraday[['date','close_spx','close_vix']].set_index('date').plot(figsize = (20,10), secondary_y = 'close_vix')
latest_intraday_stats = day_block_stats(index_intraday)
latest_intraday_stats['SPX_trend'] = latest_intraday_stats['SPX_OLS_Coeff']*latest_intraday_stats['SPX_OLS_R2']
latest_intraday_stats['VIX_trend'] = latest_intraday_stats['VIX_OLS_Coeff']*latest_intraday_stats['VIX_OLS_R2']
latest_intraday_stats['SPX_Forecast'] = np.nan

for idx, row in latest_intraday_stats.iterrows():
    curr_estimates = reg_estimates[reg_estimates.Block_Label == row.Block_Label]
    curr_estimates = curr_estimates.join(row).iloc[:,[0,3]]
    curr_estimates.columns = ['Coeff', 'Vals']
    latest_intraday_stats.loc[idx,'SPX_Forecast'] = sum(curr_estimates.Coeff*curr_estimates.Vals)

latest_intraday_stats = latest_intraday_stats.T
latest_intraday_stats.columns = [time_block_strtoactual(i + 1) + " - " + time_block_strtoactual(i + 2) for i in latest_intraday_stats.columns.tolist()]
latest_intraday_stats.iloc[1:,:]


,09:30 - 10:00,10:00 - 10:30,10:30 - 11:00,11:00 - 11:30,11:30 - 12:00
SPX_OLS_Coeff,-0.0242,0.0017,0.0795,0.1500,-0.0491
SPX_OLS_R2,0.0280,-0.0310,0.6260,0.8870,0.7290
SPX_VIX_corr,-0.8222,-0.5383,-0.8654,-0.1034,0.2937
SPX_adf_sig,0.6949,0.0160,0.9304,0.9198,0.8088
SPX_bar_std,0.0003,0.0002,0.0002,0.0002,0.0002
SPX_block_return,0.0005,0.0001,0.0010,0.0016,-0.0004
SPX_block_std,0.0003,0.0003,0.0002,0.0002,0.0002
VIX_OLS_Coeff,-0.0077,-0.0021,0.0001,0.0001,-0.0002
VIX_OLS_R2,0.9020,0.4950,-0.0320,-0.0320,-0.0380
VIX_adf_sig,0.8047,0.0000,0.7872,0.8784,0.5011


In [204]:
expiry_index = 0
rowshift = 3
show_weeklies = True

put_spreads, evs = get_spreads(puts, curr_spx, curr_vix, curr_skew, expiry_index, rowshift)
filtered_ps = put_spreads[(put_spreads.Credit > 0.5) & 
                          (abs(put_spreads.Short_Delta) < 0.6)].sort_values('EV', ascending = False)

if show_weeklies:
    commissions = 0.0266
    max_contracts = 10
    filtered_ps = []
    
    for rowshift in range(1,6):
        
        put_spreads, evs = get_spreads(puts, curr_spx, curr_vix, curr_skew, expiry_index, rowshift)
        put_spreads = put_spreads[(put_spreads.Credit > 0.5) & 
                                  (abs(put_spreads.Short_Delta) < 0.6)].sort_values('EV', ascending = False)
        filtered_ps.append(put_spreads)
    
    filtered_ps = pd.concat(filtered_ps, axis = 0).reset_index(drop = True)
    
    atm_puts = puts[(puts.Expiry == filtered_ps.reset_index(drop = True).loc[0, 'Expiry']) &
                    (abs((abs(puts.modelDelta) - 0.5)) <= 0.05)][['Expiry','Ask','Bid',
                                                                  'Strike','Type',
                                                                  'askDelta','bidDelta',
                                                                  'modelDelta']].sort_values('modelDelta', 
                                                                                             ascending = False).reset_index(drop = True)
    atm_delta = atm_puts.loc[0, 'modelDelta']
    atm_credit = atm_puts.loc[0, 'Bid']

    weekly_ps = filtered_ps.copy().reset_index(drop = True)
    weekly_ps['ContractsForATMEquiv'] = round(pd.to_numeric(atm_delta/weekly_ps.Delta),0)

    conversion_columns = ['Delta','Gamma','Theta','Vega','Credit','MaxLoss','EV']

    for col in conversion_columns:
        weekly_ps[col] = weekly_ps[col]*weekly_ps.ContractsForATMEquiv
    
    weekly_ps = weekly_ps[(weekly_ps.Credit >= atm_credit + weekly_ps.ContractsForATMEquiv*commissions) &
                          (weekly_ps.ContractsForATMEquiv <= max_contracts)].reset_index(drop = True)
    
weekly_ps.sort_values('MaxLoss', ascending = False)

#filtered_ps

,Short_Strike,Long_Strike,Short_Delta,Delta,Gamma,Theta,Vega,Credit,MaxLoss,BreakEven,EV,Win Prob,Expiry,ContractsForATMEquiv
6,"2,800.0000","2,785.0000",-0.5917,-0.4329,0.0016,0.3951,-0.0834,19.4202,"-2,557.9800","2,793.5266",-3.4945,0.4834,2019-03-01,3.0000
2,"2,800.0000","2,790.0000",-0.5917,-0.4913,0.0013,0.5469,-0.2058,21.8670,"-2,813.3000","2,795.6266",-4.4886,0.4683,2019-03-01,5.0000
0,"2,800.0000","2,795.0000",-0.5917,-0.5055,0.0015,0.7541,-0.3307,20.7340,"-2,926.6000","2,797.9266",-6.5152,0.4519,2019-03-01,10.0000
3,"2,795.0000","2,785.0000",-0.5423,-0.4744,0.0030,0.4200,0.0241,19.8670,"-3,013.3000","2,791.0266",-4.6995,0.5013,2019-03-01,5.0000
16,"2,800.0000","2,775.0000",-0.5917,-0.4582,0.0026,0.2593,0.1043,19.5468,"-3,045.3200","2,790.2266",-4.1348,0.5070,2019-03-01,2.0000
1,"2,795.0000","2,790.0000",-0.5423,-0.4885,0.0033,0.6169,-0.0855,18.7340,"-3,126.6000","2,793.1266",-6.7278,0.4862,2019-03-01,10.0000
4,"2,790.0000","2,780.0000",-0.4934,-0.4488,0.0039,0.2431,0.2275,17.3670,"-3,263.3000","2,786.5266",-5.4137,0.5335,2019-03-01,5.0000
17,"2,795.0000","2,770.0000",-0.5423,-0.4355,0.0038,0.1268,0.2894,17.3468,"-3,265.3200","2,786.3266",-4.5685,0.5349,2019-03-01,2.0000
11,"2,800.0000","2,780.0000",-0.5917,-0.5641,0.0026,0.4127,0.0130,24.8202,"-3,517.9800","2,791.7266",-4.6617,0.4963,2019-03-01,3.0000
7,"2,795.0000","2,780.0000",-0.5423,-0.5545,0.0038,0.3595,0.1478,23.0936,"-3,690.6400","2,789.2266",-5.3158,0.5142,2019-03-01,4.0000


In [205]:
call_spreads, evs = get_spreads(calls, curr_spx, curr_vix, curr_skew, expiry_index, rowshift)
filtered_cs = call_spreads[(call_spreads.Credit > 0.5) & 
                           (abs(call_spreads.Short_Delta) < 0.6)].sort_values('EV', ascending = False)
filtered_cs

,Short_Strike,Long_Strike,Short_Delta,Delta,Gamma,Theta,Vega,Credit,MaxLoss,BreakEven,EV,Win Prob,Expiry
8,"2,780.0000","2,805.0000",0.5982,0.2369,-0.0003,-0.1528,0.0476,13.1734,"-1,182.6600","2,793.1734",0.9026,0.5140,2019-03-01
9,"2,785.0000","2,810.0000",0.5536,0.2394,0.0004,-0.2245,0.1584,11.8734,"-1,312.6600","2,796.8734",0.4891,0.5405,2019-03-01
10,"2,790.0000","2,815.0000",0.5065,0.2383,0.0012,-0.3019,0.2711,10.5734,"-1,442.6600","2,800.5734",0.0668,0.5668,2019-03-01
11,"2,795.0000","2,820.0000",0.4580,0.2320,0.0019,-0.3630,0.3757,9.2734,"-1,572.6600","2,804.2734",-0.3711,0.5928,2019-03-01
12,"2,800.0000","2,825.0000",0.4083,0.2215,0.0027,-0.4111,0.4685,8.0734,"-1,692.6600","2,808.0734",-0.7310,0.6191,2019-03-01
13,"2,805.0000","2,830.0000",0.3587,0.2073,0.0033,-0.4481,0.5437,6.8734,"-1,812.6600","2,811.8734",-1.1189,0.6448,2019-03-01
14,"2,810.0000","2,835.0000",0.3105,0.1898,0.0037,-0.4696,0.5953,5.7734,"-1,922.6600","2,815.7734",-1.4399,0.6705,2019-03-01
15,"2,815.0000","2,840.0000",0.2635,0.1680,0.0040,-0.4618,0.6135,4.7234,"-2,027.6600","2,819.7234",-1.7487,0.6958,2019-03-01
16,"2,820.0000","2,845.0000",0.2202,0.1467,0.0041,-0.4484,0.6130,3.7734,"-2,122.6600","2,823.7734",-1.9989,0.7208,2019-03-01
